In [8]:
import numpy as np
import tensorflow as tf
from lucid.optvis import objectives
from lucid.optvis.param import images
from lucid.optvis import render
import lucid.modelzoo.vision_models as models
import lucid.misc.io.loading as loading
import lucid.misc.io.saving as saving
import lucid.misc.io.showing as showing
import lucid.optvis.param  as param

In [2]:
v3=models.inceptionV3()
v3.load_graphdef()

In [3]:
def get_filer_numbers(layers):
    
    im=tf.placeholder(tf.float32,shape=(1,229,229,3),name='im')
    T=render.import_model(v3,im,im)
    
    filter_list=[]
    
    for l in layers:
        acct=T(l)
        filter_list.append(acct.shape[-1].value)
    
    return filter_list       

In [4]:
tf.reset_default_graph()
layers=['InceptionV3/InceptionV3/Mixed_6d/concat_v2',
 'InceptionV3/InceptionV3/Mixed_6e/concat_v2',
 'InceptionV3/InceptionV3/Mixed_7a/concat_v2',
 'InceptionV3/InceptionV3/Mixed_7b/Branch_1/concat_v2',
 'InceptionV3/InceptionV3/Mixed_7b/Branch_2/concat_v2',
 'InceptionV3/InceptionV3/Mixed_7b/concat_v2',
 'InceptionV3/InceptionV3/Mixed_7c/Branch_1/concat_v2',
 'InceptionV3/InceptionV3/Mixed_7c/Branch_2/concat_v2',
 'InceptionV3/InceptionV3/Mixed_7c/concat_v2']

fileter_list=get_filer_numbers(layers)

print(fileter_list)


[768, 768, 1280, 768, 768, 2048, 768, 768, 2048]


In [ ]:
tf.reset_default_graph()
channel=lambda layer,n: objectives.channel(layer=layer,n_channel=n)

for i,l in enumerate(layers):
    op_images=np.zeros((fileter_list[i],100,100,3))
    
    for filter in range(fileter_list[i]):
        with tf.Graph().as_default() as g,tf.Session() as sess:
            
            param_f=lambda : param.image(w=100)
            obj=channel(l,filter)
            T=render.make_vis_T(model=v3,objective_f=obj,param_f=param_f)
            with tf.device('/GPU:0'):
                vis_op=T('vis_op')
                tf.global_variables_initializer().run()
                for j in range(512):
                        vis_op.run()

                t_image=T('input')
                loss=T('loss')
                print('{layer}:{filter} - {loss}'.format(layer=l,filter=filter,loss=loss.eval()))

                #showing.show(t_image.eval()[0])
                op_images[filter]=t_image.eval()[0]
    with open("D:\\InceptionV3\\spritemap\\"+l.replace('/','_')+'.npy','wb') as f:
        saving.save(op_images,f)
    del op_images
   